In [1]:
import pandas as pd
import numpy as np

train_cn = pd.read_excel('汽车领域多语种迁移学习挑战赛初赛公开数据_A榜/汽车领域多语种迁移学习挑战赛初赛训练集/中文_trian.xlsx')
train_ja = pd.read_excel('汽车领域多语种迁移学习挑战赛初赛公开数据_A榜/汽车领域多语种迁移学习挑战赛初赛训练集/日语_train.xlsx')
train_en = pd.read_excel('汽车领域多语种迁移学习挑战赛初赛公开数据_A榜/汽车领域多语种迁移学习挑战赛初赛训练集/英文_train.xlsx')

test_ja = pd.read_excel('汽车领域多语种迁移学习挑战赛初赛公开数据_A榜/testA.xlsx', sheet_name='日语_testA')
test_en = pd.read_excel('汽车领域多语种迁移学习挑战赛初赛公开数据_A榜/testA.xlsx', sheet_name='英文_testA')

In [49]:
train_cn[~train_cn['槽值1'].isnull()].head(2)

,原始文本,意图,槽值1,槽值2
0,16.5度,adjust_ac_temperature_to_number,offset:16.5,NaN
1,16度,adjust_ac_temperature_to_number,offset:16,NaN


In [48]:
train_ja[~train_ja['槽值1'].isnull()].head(2)

,原始文本,中文翻译,意图,槽值1,槽值2
35,エアコンの冷房モードをオンにして,开启空调的制冷模式,open_ac_mode,mode:冷房モード,NaN
36,エアコンの冷房モードをつけて,开启空调的制冷模式,open_ac_mode,mode:冷房モード,NaN


In [37]:
train_cn['原始文本'].apply(len).max(), train_ja['原始文本'].apply(len).max(), train_en['原始文本'].apply(len).max()

(36, 33, 71)

In [43]:
s = '菅田将暉のバースデイイブを再生してくれ'
tag1 = '菅田将暉'

In [46]:
ner_label = np.zeros(len(s))
ner_label[s.index(tag1): s.index(tag1) + len(tag1)] = 1
ner_label[s.index(tag2): s.index(tag2) + len(tag2)] = 2

In [47]:
ner_label

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

# transformer基础

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# AutoTokenizer：分词器

# Auto：自动识别的
model_name = "bert-base-chinese"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [144]:
inputs = tokenizer(["阿水很帅，我也这样觉得。", "不对啊，你在欺骗我"], truncation=True, max_length=20, padding=True)
inputs
# input_ids：这个字在vocab次序
# token_type_ids：字符是第一个句子的，还是第二个句子的
# attention_mask：字符是不是padding的？

{'input_ids': [[101, 8204, 4867, 3767, 3602, 10064, 3976, 2135, 7700, 4577, 7162, 3775, 1882, 102], [101, 2080, 3442, 2892, 10064, 2262, 3031, 4778, 8576, 3976, 102, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]}

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, BertModel, AutoModel
model = AutoModel.from_pretrained("hfl/chinese-roberta-wwm-ext")

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
import torch
item = {key: torch.tensor(inputs[key]).view(1, -1) for key, val in inputs.items()}

output = model(input_ids = item['input_ids'], attention_mask = item['attention_mask'])
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 2.8968e-01,  2.8604e-01,  1.7832e-01,  ..., -3.5887e-01,
           1.9458e-02, -3.0741e-02],
         [ 5.2763e-02,  4.7332e-01,  6.1362e-01,  ...,  4.0184e-01,
           9.5352e-02, -8.2413e-01],
         [ 7.3144e-01,  1.6262e-01,  8.1929e-02,  ...,  2.2394e-01,
          -4.6239e-01,  1.6442e-04],
         ...,
         [ 2.5093e+00,  6.9676e-01,  7.8148e-01,  ..., -3.6061e-02,
           9.9588e-02, -3.1432e-01],
         [ 2.1131e+00,  2.4205e-01, -1.4203e-01,  ..., -1.3002e+00,
           4.6712e-01, -2.1039e-01],
         [ 2.8968e-01,  2.8604e-01,  1.7832e-01,  ..., -3.5887e-01,
           1.9458e-02, -3.0741e-02]]], grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 9.9662e-01,  9.6177e-01,  9.8323e-01,  9.5575e-01,  7.7640e-01,
          2.6194e-01, -9.2609e-01, -3.0311e-01,  4.8948e-01, -9.8587e-01,
          9.9946e-01,  9.8744e-01, -6.3099e-01, -8.3612e-01,  9.5591e-01,
         -9.9262e-0

In [29]:
output.last_hidden_state.shape, output.pooler_output.shape

(torch.Size([1, 24, 768]), torch.Size([1, 768]))

# 意图分类模型

In [55]:
for tag in ['intent', 'device', 'mode', 'offset', 'endloc', 'landmark', 'singer', 'song']:
    train_ja['槽值1'] = train_ja['槽值1'].str.replace(f'{tag}:', '')
    train_ja['槽值2'] = train_ja['槽值2'].str.replace(f'{tag}:', '')
    
    train_cn['槽值1'] = train_cn['槽值1'].str.replace(f'{tag}:', '')
    train_cn['槽值2'] = train_cn['槽值2'].str.replace(f'{tag}:', '')

    train_en['槽值1'] = train_en['槽值1'].str.replace(f'{tag}:', '')
    train_en['槽值2'] = train_en['槽值2'].str.replace(f'{tag}:', '')

In [126]:
train_df = pd.concat([
    train_ja[['原始文本', '意图', '槽值1', '槽值2']],
    train_cn[['原始文本', '意图', '槽值1', '槽值2']].sample(10000),
    train_en[['原始文本', '意图', '槽值1', '槽值2']],
],axis = 0)
train_df = train_df.sample(frac=1.0)
train_df['意图_encode'], lbl_ecode = pd.factorize(train_df['意图'])

In [127]:
len(lbl_ecode)

18

In [128]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
config = AutoConfig.from_pretrained("bert-base-multilingual-cased")

In [129]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch
from torch import nn

# 数据集读取
class XunFeiDataset(Dataset):
    def __init__(self, encodings, intent):
        self.encodings = encodings
        self.intent = intent
    
    # 读取单个样本
    def __getitem__(self, idx):        
            
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(int(self.intent[idx]))
        return item
    
    def __len__(self):
        return len(self.intent)
    
class XunFeiModel(nn.Module):
    def __init__(self, num_labels): 
        super(XunFeiModel,self).__init__() 
        self.model = model = AutoModel.from_pretrained("bert-base-multilingual-cased")
        self.dropout = nn.Dropout(0.1) 
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
        logits = self.classifier(sequence_output[:,0,:].view(-1,768))
        
        return logits

In [131]:
train_encoding = tokenizer(train_df['原始文本'].tolist()[:-500], truncation=True, padding=True, max_length=40)
val_encoding = tokenizer(train_df['原始文本'].tolist()[-500:], truncation=True, padding=True, max_length=40)

train_dataset = XunFeiDataset(train_encoding, train_df['意图_encode'].tolist()[:-500])
val_dataset = XunFeiDataset(val_encoding, train_df['意图_encode'].tolist()[-500:])

In [133]:
# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [134]:
model = XunFeiModel(18)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
model = model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [135]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

loss_fn = CrossEntropyLoss() # ingore index = -1
optim = AdamW(model.parameters(), lr=5e-5)

In [136]:
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    for batch in train_loader:
        # 正向传播
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)

        pred = model(
            input_ids, 
            attention_mask
        )
        
        loss = loss_fn(pred, label)
        
        # 反向梯度信息
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 参数更新
        optim.step()

        iter_num += 1
        
        if(iter_num % 100 == 0):
            print("iter_num: %d, loss: %.4f, %.2f%% %.4f" % (
                iter_num, loss.item(), iter_num/total_iter*100, 
                (pred.argmax(1) == label).float().data.cpu().numpy().mean(),
            ))

def validation():
    model.eval()
    label_acc = 0
    for batch in val_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label = batch['label'].to(device)

            pred = model(
                input_ids, 
                attention_mask
            )
    
            label_acc += (pred.argmax(1) == label).float().sum().item()
    
    label_acc = label_acc / len(val_dataloader.dataset)

    print("-------------------------------")
    print("Accuracy: %.4f" % (label_acc))
    print("-------------------------------")

In [137]:
for epoch in range(2):
    train()
    validation()

iter_num: 100, loss: 0.9366, 13.91% 0.7500
iter_num: 200, loss: 0.3068, 27.82% 0.8750
iter_num: 300, loss: 0.2977, 41.72% 0.9375
iter_num: 400, loss: 0.5186, 55.63% 0.8125
iter_num: 500, loss: 0.3027, 69.54% 0.8750
iter_num: 600, loss: 0.0159, 83.45% 1.0000
iter_num: 700, loss: 0.0124, 97.36% 1.0000
-------------------------------
Accuracy: 0.9240
-------------------------------
iter_num: 100, loss: 0.3482, 13.91% 0.8750
iter_num: 200, loss: 0.0354, 27.82% 1.0000
iter_num: 300, loss: 0.2516, 41.72% 0.9375
iter_num: 400, loss: 0.1396, 55.63% 0.9375
iter_num: 500, loss: 0.0116, 69.54% 1.0000
iter_num: 600, loss: 0.4513, 83.45% 0.9375
iter_num: 700, loss: 0.0085, 97.36% 1.0000
-------------------------------
Accuracy: 0.9200
-------------------------------


In [138]:
def prediction():
    model.eval()
    test_label = []
    for batch in test_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            pred = model(input_ids, attention_mask)
            test_label += list(pred.argmax(1).data.cpu().numpy())
    return test_label

In [139]:
test_encoding = tokenizer(test_en['原始文本'].tolist(), truncation=True, padding=True, max_length=40)
test_dataset = XunFeiDataset(test_encoding, [0] * len(test_en))
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

test_en_intent = prediction()

test_encoding = tokenizer(test_ja['原始文本'].tolist(), truncation=True, padding=True, max_length=40)
test_dataset = XunFeiDataset(test_encoding, [0] * len(test_ja))
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

test_ja_intent = prediction()

In [140]:
test_ja['意图'] = [lbl_ecode[x] for x in test_ja_intent]
test_en['意图'] = [lbl_ecode[x] for x in test_en_intent]
test_en['槽值1'] = np.nan
test_en['槽值2'] = np.nan

test_ja['槽值1'] = np.nan
test_ja['槽值2'] = np.nan

writer = pd.ExcelWriter('submit.xlsx')
test_en[['意图', '槽值1', '槽值2']].to_excel(writer, sheet_name='英文_testA', index=None)
test_ja[['意图', '槽值1', '槽值2']].to_excel(writer, sheet_name='日语_testA', index=None)
writer.save()
writer.close()

# 槽值识别模型

In [ ]:
# 数据集读取
class XunFeiDataset2(Dataset):
    def __init__(self, encodings, label_i, label_j):
        self.encodings = encodings
        self.intent = intent
        self.tag1 = tag1
        self.tag2 = tag2
    
    # 读取单个样本
    def __getitem__(self, idx):        
        
        tags_single = [0] + list(tags_single) + [0]
        tags_single = tags_single + [0] * (32 - len(tags_single))
        tags_single = tags_single[:maxlen]
            
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label_i'] = torch.tensor(int(self.label_i[idx]))
        item['label_j'] = torch.tensor(int(self.label_j[idx]))
        return item
    
    def __len__(self):
        return len(self.label_i)
    
class XunFeiModel2(nn.Module):
    def __init__(self, num_labels_i, num_labels_j): 
        super(XunFeiModel,self).__init__() 

        #Load Model with given checkpoint and extract its body
        self.model = AutoModel.from_pretrained("hfl/chinese-roberta-wwm-ext")
        self.dropout = nn.Dropout(0.1) 
        self.classifier_i = nn.Linear(768, num_labels_i)
        self.classifier_j = nn.Linear(768, num_labels_j)

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
        
        # 问诊方向
        logits_i = self.classifier_i(sequence_output[:,0,:].view(-1,768))
        # 疾病标签
        logits_j = self.classifier_j(sequence_output[:,0,:].view(-1,768))
        
        return logits_i, logits_j